# NHibernate and C\# programming

## Wprowadzenie

1. Co to jest NHibernate

NHibernate jest frameworkiem do mapowanie obiektów z bazy danych na obiekty w kodzie programu. Kod jest bardziej odporny na błędy typograficzne oraz łatwiejszy do zmian. Dodatkowo dzięki *LINQ* bardzo łatwo można tworzyć skomplikowane zapytania bazy danych bez konieczności ich znajomości. O wiele łatwiej również zmienić bazę danę (nie wymaga to aż tak dużej liczby zmian).

Alternatywą może być Dapper lub EntityFramework.

Podstawową monomenklaturę stanowi:

1. Dialekty 
2. Sesja
3. Fluent nHiberante
4. Klasa i mapowanie do klasy
5. Repozytorium

## Kofiguracja początkowa (boilerplate)

*NHibernate* nakłada pewien narzut związany z dodatkowym kodem, który musi zostać napisany ze względu na konkretne zastosowanie. Przykładem jest tutaj zarządzanie sesją. W zależności od tego czy *ORM* zostanie użyty w silniku do tworzenia stron czy też w aplikacji desktopowej (gruby klient), to programista musi zdecydować kto i jak ma zwalniać sesję. 

W pierwszym kroku należy zainstalować dwie paczki, jedna do `FluentNHibernate` i druga `System.Data.SQLite`. Pierwszy zawiera wszystkie niezbędne klasy do mapowania klas oraz budowania zapytań w konkretnym dialekcie języka SQL. Drugi zawiera niezbędne klasy do nawiązywania i utrzymywania połączenia z bazą danych. Ponieważ przykłady powstały w `csi` (trybie interaktywnym języka C#), zostało użyte polecenie `#r` do instalacji. W środowisku Visual Studio/ Visual Studio Code, należy użyć narzędzia `Nuget`. Jednak nazwy bibliotek pozostają takie same.

In [1]:
#r "nuget: FluentNHibernate, 3.1.0"
#r "nuget: System.Data.SQLite, 1.0.113.7"

Installed Packages FluentNHibernate, 3.1.0 System.Data.SQLite, 1.0.113.7

Poniżej zostanie przedstawiony kod, który pozwala utworzyć tabele na podstawie przekazanych mapowań. Jest to szczególnie przydatne w testach jednostkowych, gdzie bazę tworzy się na żądanie. Nie jest to z kolei wymagane, gdy używamy istniejacego schematu lub chcemy ręcznie utworzyć całą strukturę.

In [2]:
using NHibernate.Cfg;
using NHibernate.Tool.hbm2ddl;

private static void BuildSchema(Configuration config)
{
    // this NHibernate tool takes a configuration (with mapping info in)
    // and exports a database schema from it
    new SchemaExport(config)
            .Create(false, true);
}

Mając funkcję do tworzenia schematu bazy danych można przystąpić do tworzenia funkcji tworzącej sesje tzw. fabryka sesji. Ze wzgędu na to, że kod ma być uruchamiany w Jupyter Notebooku, typy do mapowania mogą znajdowąć się w wielu modułach. Dlatego typ mapy należy przekazać ręcznie. W pozostałych przypadkach wystarczy użyć `m.FluentMappings.AddFromAssemblyOf<Program>()` dla aplikacji desktopowej lub w zamian za `Program` inny typ, który znajduje się w przestrzeni nazw. Zaszyty mechanizm wbudowany w bibliotekę, będzie próbował dodać każdy typ zawierający klasę do mapowania do schematu mapowania klas na tabele (za pomocą mechanizm *refleksji*). Ze względu, że w trybie interaktywnym, klasy mogą być tworzone dynamicznie załadowanie na początku wszystkich klas nie pomoże. W konfiguracji zaszyty zostały również sposób przechowywania danych w pamięci, co jest wygodne w przypadku pracy z Jupyter Notebookiem. Jeśli po restarcie aplikacji wymagany jest dostęp do poprzednio wykonanych operacji, należy odkomentować kod `.UsingFile("database.sqlite")`.

In [3]:
using FluentNHibernate.Cfg;
using FluentNHibernate.Cfg.Db;
using NHibernate;

private static ISessionFactory CreateSessionFactory(params Type[] mappingTypes)
{
    return Fluently.Configure()
                        .Database(SQLiteConfiguration.Standard
                            .UsingFile("database.sqlite"))
                        .Mappings(m => {
                            foreach(var mappingType in mappingTypes) 
                                m.FluentMappings.Add(mappingType);
                        })
                        .ExposeConfiguration(BuildSchema)//Call of ours BuildSchema
                        .BuildSessionFactory();
}

W powyższym przykładzie został użyty sterownik dla bazy danych `Sqlite`. Na tej podstawie zostały użyty odpowiedni dialekt, który umożliwia budowanie składni konkretnej bazy danych. Przykładowo Microsoft SQL używa języka *T-SQL*, Oracle *PL/SQL* itd. Wszystkie one wywodzą się ze standardy `Ansi SQL`, ale mają specyficzne dla siebie elementy jak np. `top` w *T-SQL*. Przykładem innego silnika może być: `PostgreSQLConfiguration`. Powyższy przykład prezentuje sposób budowania konfiguracji za pomocą notacji typu *fluent*, gdzie konfigurację dodaje za pomocą kropek (każdy poprzedni element dodaje coś do konfiguracji i przekazuje w rezultacie tą samą instancję klasy).

W kolejnym przykładzie zostanie zadeklarowany obiekt, który będzie mógł być użyty w mapowaniu. Należy zwrócić uwage, że klasy muszą zawierać właściwości wirtulne. `NHibernate` dodaje swoją implementację pól, dzięki czemu wiadomo, które pola zostały zmienione i muszą być zapisane do bazy danych w celu synchroniacji danych.

In [4]:
public class Employee
{
  public virtual int Id { get; protected set; }
  public virtual string FirstName { get; set; }
  public virtual string LastName { get; set; }

  public override string ToString() => $"Id: {Id}, {FirstName}, {LastName}";
}

Klasa może zawierać dowolnie dużo funkcji ze względu na to, że `NHibernate` operuje na poziomie samych właściwości. Niemniej jednak należy pamiętać o zasadzie segregacji obowiązków i pojedyńczej odpowiedzialności. Kolejny przykład przedstawia sposób w jaki można mapować pola naszej klasy z danymi w bazie danych.

In [5]:
using FluentNHibernate.Mapping;

public class EmployeeMap : ClassMap<Employee>
{
  public EmployeeMap()
  {
    Table("Employee");

    Id(x => x.Id);
    
    Map(x => x.FirstName);
    Map(x => x.LastName);
  }
}

W powyższym przykładzie klasa `ClassMap` to typ generyczny, który jako argument zawiera typ mapowanego obiektu. W konstrukturze następuje właściwe mapowanie. Funkcja `Table` klasy ClassMap. Wskazuje ona nazwę tabeli skojarzoną z instancją klasy `Employee`. Funkcja `Id` wskazuje jednoznacznie identyfikator obiektu (klucz główny). Sprawdzenie czy dwa obiekty są takie same następuje właśnie po tym polu. Dodatkowo w momencie generowania schematu (funkcja `BuildSchema`) pole to zostanie oznaczone jako klucz główny w bazie danych. W przypadku złożonych kluczy głównych w bazie danych należy przedefiniować w klasie `Employee` funkcje `GetHashCode` i `Equals`. W ten sposób *NHiberante* jest w stanie porównać ze sobą dwie instancje tej samej klasy. Funkcje `Map` i `Id` zwracają obiekt, który można dodatkowo skonfigurować. 

* Funkcja `Id`.
    - `GeneratedBy.Identity()` w trakcie tworzenia nowego obiektu identyfikator będzie automatycznie inkrementowany przy zapisanie do bazy danych.
    - `GeneratedBy.Sequence("SequenceId")` w trakcie tworzenia nowego obiektu identyfikator zostanie przypisany na podstawie wartości z sekwencji o nazwie `SequenceId`, która musi istnieć w bazie danych.
    - `Column("ID")` pozwala na zmianę nazwy mapowanej kolumny z bazy danych. 
* Funkcja `Map`
    - `Default("defaultValue")` jeśli nie zostanie przekazana żadna wartość w trakcie tworzenia obiektu `defaultValue` zostanie użyta.
    - `CustomSqlType("varchar(max)")` wymusza stosowanie innego typu, niż domyślnie zdefiniowany w dialekcie.
    - `Length(32)` określa typu (najczęściej) łańcucha znaków w bazie danych.
    - `Check("len(columnName) > 1")` pozwala utworzyć w bazie danych *constraint* w języku *SQL* nakładający ograniczenia na kolumnę lub grupę kolumn.
    - `Nullable` i `Not.Nullable()` określa czy pole `Null` (C\#) / `NULL` (DB) jest dopuszczalne. W trakcie tworzenia schematu pole to będzie ozaczone jako *nullable* lub *mandatory*. W trakcie operacji `insert` lub `update` pole to jest sprawdzane w kontekście występowania wartości dla tej właściwości.
    - `Index("column_idx")` łączy jedno lub więcej pół (na podstawie nazwy indeksu) w jeden indeks w bazie danych. Pole to jest używane w momencie budowania schematu dla tabeli.
    - `Unique` i `UniqueKey` tworzy indeks unikalny dla pola lub wielu pól. Funkcje na dwie deklarancje bezparametrową `Unique`, unikalność kolumny dotyczy tylko jednej kolumn lub `UniqueKey` z nazwą indeksu unikalnego, który dotyczy wielu pól.
    - `Formula("SQL expression")` zamiast zwykłej kolumny w bazie danych utworzona zostanie `Computed column`. Silnik bazy danych w zależności od implementacji zwróci wartość tej kolumny dopiero w momencie wykonywania zapytania `select` lub po operacji `insert` / `update` (`persistent store`).
    - `ReadOnly()` nie pozwala na modyfikację wartości kolumny. Szczególnie często tej funkcji używa się w tabelach przechowujących słowniki.
    - `Update()` i `Insert()` w parze z `Not.Update()` pozwala kontrolować czy wartość pola powinna być przekazywana w trakcie wykonywania operacji `Update`/ `Insert`. Przykłałem może być tutaj kolumna (właściwość) `CREATEDATE` (`CreateDate`), której wartość powinna być ustawiana tylko raz w trakcie dodawania do bazy danych.

Poniżej znajduje się przykład zastosowania dodatkowych konfiguracji dla obu funkcji.    

In [6]:
using FluentNHibernate.Mapping;

public class EmployeeMap : ClassMap<Employee>
{
  public EmployeeMap()
  {
    Table("Employee");

    Id(x => x.Id)
        .GeneratedBy.Identity();

    Map(x => x.FirstName)
        .Length(64)
        .Not.Nullable();
    Map(x => x.LastName)
        .Length(64)
        .Not.Nullable();
  }
}

Po utworzeniu czystej klasy przechowującej dane, po stworzeniu mapowania i konfiguracji bazy danych można rozpocząć wykonywanie operacji na obiektach. Istnieją dwie metody klasy `Session` do zapisywania instancji obiektów `Save` i `SaveOrUpdate`. Pierwsza służy do wymuszenia operacji `Insert` druga (`SaveOrUpdate`) sprawdza czy klasa została zainicjowana samodzielnie przez programistę czy zwrócona z bazy danych. W pierwszym przypadku wykonywana jest operacja `Insert` w drugim `Update`. Poniżej znajduje się przykład zapisu do bazy danych dwóch pracowników (`Employee`) oraz ich wyświetlenie przy użyciu *lambda expression*.

In [7]:
using System.Data.SQLite;

var sessionFactory = CreateSessionFactory(typeof(EmployeeMap));

using (var session = sessionFactory.OpenSession())
{
    using (var transaction = session.BeginTransaction()) //initialize transaction
    {        
        var mike = new Employee() { FirstName = "Mike", LastName = "Harrison" };
        session.Save(mike);

        var jack = new Employee() { FirstName = "Jack", LastName = "Kowalski" };
        session.SaveOrUpdate(jack); //or just Save        

        transaction.Commit();
        
        Console.WriteLine($"Mike Harrison Id: {mike.Id}");
        Console.WriteLine($"Jack Kowalski Id: {jack.Id}");
    }

    session.Query<Employee>().ToList().ForEach(e=> Console.WriteLine(e));
}        

Mike Harrison Id: 1
Jack Kowalski Id: 2
Id: 1, Mike, Harrison
Id: 2, Jack, Kowalski


W przykładzie została użyta transakcja, która nie jest wymagana w przypadku, gdy wykonujemy jedną operację (pojedyncze operacje jak `insert`, `update` czy `delete` są z definicji atomowe w SQL). Należy w tym miejscu zauważyć, że właściwość `Id` nie zostało nigdzie przekazane w czasie tworzenia. Baza danych uzupełniła je automatycznie. Po operacji `Save` i analogicznie po operacji `SaveOrUpdate` wartość ta została przypisana do obiektu (instancje `mike` i `jack` zostały zmienione).

## Mapowanie relacji

Zależności miedzy poszczególnymi tabeli określają relacje między nimi. Są one nieodłącznym elementem każdej relacyjnej bazy danych. Dostępne są trzy rodzaje relacji.

* Jeden do jednego - jedenemu wierszowi odpowiada jeden wiersz w powiązanej tabeli. Przykładem może być pracownik i jego numer konta.
* Jeden do wielu - jednemu wierszowi odpowiada wiele powiązanych wierszy powiązanej tabeli. Przykładem może być pracownik i jego stanowisko (np. programista). Jeden pracownik posiada jedno stanowisko, ale to samo stanowisko może należeć do wielu pracowników.
* Wiele do wielu - wymaga dodatkowej trzeciej tabeli wiążącej obie tabele (encja wtrącona). Analogicznie do poprzednich przykładów pracownik może mieć wiele kont email (osobiste z jego imieniem i nazwiskiem oraz *supportowe* na które zgłaszane są błędy z działaniem oprogramowania), ostatnie konto może być używane przez wielu pracowników z tego samego działu.

Poniżej znajduje się sposób budowania każdego rodzaju mapowania.

### Jeden do jednego

Zakładamy, że jeden adres można jednoznacznie przypisać do jednego adresu.

In [8]:
public class Address
{
    public virtual int Id { get; set; }

    public virtual string City { get; set; }

    public virtual string Street { get; set; }

    public virtual string Number { get; set; }

    public virtual string Region { get; set; }

    public virtual string RegionCode { get; set; }
}

public class Bank
{
    public virtual int Id { get; protected set; }

    public virtual string Name { get; set; }

    public virtual Address Address { get; set; }    
}

public class AddressMap : ClassMap<Address>
{
    public AddressMap()
    {
        Table("Address");

        Id(x => x.Id).GeneratedBy.Identity();

        Map(x => x.City)
            .Length(32)
            .UniqueKey("AddressStreetNumberRegion");
        Map(x => x.Street)
            .Length(256)
            .UniqueKey("AddressStreetNumberRegion");
        Map(x => x.Number)
            .Length(5)
            .UniqueKey("AddressStreetNumberRegion");
        Map(x => x.Region)
            .Length(32)
            .UniqueKey("AddressStreetNumberRegion");                
        
        Map(x => x.RegionCode);
    }
}

public class BankMap : ClassMap<Bank>
{
    public BankMap()
    {
        Id(x => x.Id);

        Map(x => x.Name)
            .Length(128)
            .Unique();        

        HasOne(x => x.Address)
            .Cascade.All();
    }
}

W celu dodania odwołania do danych innej tabeli (w kodzie innej kolekcji) należy użyć w funkcji `HasOne` wskazując jednocześnie pole do jakiego się odwołujemy. W deklaracji typu znajduje się informacja do jakiej kolecji referencja się odwołuje. Typ ten również musi zawierać własne mapowanie. W ten sposób `NHibernate` będzie w stanie znaleźć w przypadku banku jego adres oraz w adresie odpowiadający mu bank. Konfiguracja `Cascade.All()` pozwalają określi sposób tworzenia i usuwania obiektów pośrednich. Za każdym razem, gdy usuwamy `Bank`, usunięte zostanie również odpowiadający mu `Address`. Analogicznie w przypadku dodawania nowej instancji klasy `Bank`.

Poniżej znajduje się przykład dodawania pól.

In [9]:
using System.Data.SQLite;

var sessionFactory = CreateSessionFactory(typeof(AddressMap), typeof(BankMap));

using (var session = sessionFactory.OpenSession())
{
    using (var transaction = session.BeginTransaction()) //iniialize transaction
    {
        Bank bank = new Bank {
            Name = "DummyBank",
            Address = new Address {
                City = "Kraków",
                Street = "Krakowska",
                Number = "3/15",
                Region = "Małopolska",
                RegionCode = "31-876"
            }
        };    

        session.Save(bank); //all related class will be inserted to DB    
        
        Address address2 = new Address {
            City = "Kraków",
            Street = "Warszawska",
            Number = "5",
            Region = "Małopolska"
        };

        session.Save(address2);
        
        Bank bank2 = new Bank {
            Name = "Dummy bank 2",
            Address = session.Query<Address>()
            .Where(x => x.Street == "Warszawska" && x.Number == "5")
            .SingleOrDefault(),
        };
        
        session.Save(bank2);

        transaction.Commit();
    }

    var banks = session.Query<Bank>();

    foreach(var bank in banks){
        Console.WriteLine($"At address {bank.Address?.Street} is located the following bank {bank.Name}");
    }
    
    session.Query<Address>().ToList().ForEach(a => Console.WriteLine($"{a.Street} {a.Number}, {a.City} ({a.Region})"));
}        

At address Krakowska is located the following bank DummyBank
At address Warszawska is located the following bank Dummy bank 2
Krakowska 3/15, Kraków (Małopolska)
Warszawska 5, Kraków (Małopolska)


Konstrukcja `bank.Address?.Street` pozwala zwrócić `Null` w przypadky, gdy adres nie został przekazany. Pozwala to uniknąć w tym przypadku błędu wykonywania *Null reference exception*. Funkcja `SingleOrDefault()` zwraca jeden wynik lub `Null`. Inną opcją jest użycie `Single`, wtedy wyjątek zostanie rzucony w przypadku braku jakiegokolwiek wyniku.

### Jeden do wielu

Relacja tę mapuje się w kodzie za pomocą kolekcji. Załóżmy, że każde konto jest przypisane do jednego banku, a jeden bank może mieć wiele kont. Poniższy przykład prezentuje sposób, w jaki można mapować taką relację.

In [10]:
public class Account
{
    public virtual int Id { get; protected set; }

    public virtual string Number { get; set; }   

    public virtual Bank BankDetails { get; set; }                
}

public class Bank
{
    public virtual int Id { get; protected set; }

    public virtual string Name { get; set; }

    public virtual Address Address { get; set; }
    
    public virtual IList<Account> Accounts { get; set; }
}

public class AccountMap : ClassMap<Account>
{
    public AccountMap()
    {
        Id(x => x.Id);

        Map(x => x.Number)
            .Length(16)
            .ReadOnly();

        References(x => x.BankDetails).LazyLoad();
    }
}

public class BankMap : ClassMap<Bank>
{
    public BankMap()
    {
        Id(x => x.Id);

        Map(x => x.Name)
            .Length(128)
            .Unique();        

        HasOne(x => x.Address)
            .Cascade.All();

        HasMany(x => x.Accounts)            
            .Inverse();
    }
}

Funkcja `LazyLoad` pozwala na załadowanie informacji z bazy danych w momencie dostępu do tego pola. Bez bezpośredniego odwołania wartość ta nie będzie przekazana, co powala zaoszczędzić zasoby komputera. Rzeczywistym typem właściwości `BankDetails` będzie typ `Proxy`, który dziedziczy po typie `Bank`, co pozwala na realizację dynamicznego ładowania danych. W relacji `Bank` i `Account` nie ma zdefiniowanego nazwy klucza obcego (kolumny). Tak zdefiniowana realizacja powoduje, że połączenie bazuje na kluczu głównym tabeli `Bank`. W przypadku, gdy odwołanie dotyczy innej kolumny tabeli `Bank` należy użyć funkcji `KeyColumn` do ręcznego określenia kolumny identyfikującej konretny wiersz w tabeli `Bank`. Kolejny przykład przedstawia sposób użycia mapowania klas.

In [11]:
using System.Data.SQLite;

var sessionFactory = CreateSessionFactory(typeof(AddressMap), typeof(BankMap), typeof(AccountMap));

using (var session = sessionFactory.OpenSession())
{
    using (var transaction = session.BeginTransaction()) //iniialize transaction
    {
        Bank bank = new Bank {
            Name = "DummyBank",
            Address = new Address {
                City = "Kraków",
                Street = "Krakowska",
                Number = "3/15",
                Region = "Małopolska",
                RegionCode = "31-877"
            },
            Accounts = new List<Account>() {
                new Account { Number = "8649747851917216" },
                new Account { Number = "1011172961443351" }
            }
        };
        
        session.Save(bank);

        transaction.Commit();
    }

    var banks = from c in session.Query<Bank>() //another option like linq
                select c;    

    foreach(var bank in banks) {
        Console.WriteLine($"Bank {bank.Name} has the followings accounts:\n{string.Join("\n", bank.Accounts.Select(a=> a.Number))}");
    }

    var accounts = from a in session.Query<Account>()
                   select new { // here we are creating new type
                        Number = a.Number,
                        BankName = a.BankDetails.Name
                   };

    foreach(var account in accounts) {
        Console.WriteLine($"Account {account.Number} belongs to {account.BankName}");
    }
}        


Bank DummyBank has the followings accounts:
8649747851917216
1011172961443351


### Wiele do wielu

Relacja jest zaimplementowana za pomocą trzech tabel (tzw. encja wtrącona). Załóżmy, że jeden pracownik może mieć możliwość rezerwacji biurka w różnych budynkach firmy, a każde biurko może być przypisane do więcej niż jednego pracownika, ze względu na częciową pracę z domu pracowników.

In [12]:
public class Desk
{
    public virtual int Id { get; set; }
    public virtual int Floor { get; set; }
    public virtual int No { get; set; }
    public override string ToString() => $"Desk: {No}, floor: {Floor}";
}

public class Department
{
    public virtual int Id { get; set; }
    public virtual string Address { get; set; }
    public virtual string Name { get; set; }
    public virtual IList<Desk> Desks { get; set; }
    public virtual IList<Employee> Employees { get; set; }    
}

public class Employee
{
  public virtual Guid Id { get; protected set; }
  public virtual string FirstName { get; set; }
  public virtual string LastName { get; set; }
  public virtual IList<Department> DepartmentAccesses {get; set; }

  public override string ToString() => $"Id: {Id}, {FirstName}, {LastName}";
}

public class DeskMap : ClassMap<Desk>
{
  public DeskMap()
  {
    Table("Desk");

    Id(x => x.Id, "DeskId") //or .Column("DeskId")
        .GeneratedBy.Identity();

    Map(x => x.Floor)
        .Not.Nullable();
    Map(x => x.No)        
        .Not.Nullable();
  }
}

public class DepartmentMap : ClassMap<Department>
{
  public DepartmentMap()
  {
    Table("Department");

    Id(x => x.Id, "DepartmentId") //or .Column("DepartmentId")
        .GeneratedBy.Identity();

    Map(x => x.Address)
        .Not.Nullable();
    Map(x => x.Name)        
        .Not.Nullable();

    HasMany(x=> x.Desks)
        .Inverse()
        .Cascade.All();

    HasManyToMany<Employee>(x => x.Employees)
        .Table("EmployeeInDepartment")
        // .ParentKeyColumn("DepartmentId")
        // .ChildKeyColumn("EmployeeId") 
        .Inverse()           
        .Cascade.All();
  }
}

public class EmployeeMap : ClassMap<Employee>
{
  public EmployeeMap()
  {
    Table("Employee");

    Id(x => x.Id, "EmployeeId")
        .GeneratedBy.Guid();

    Map(x => x.FirstName)
        .Length(64)
        .Not.Nullable();
    Map(x => x.LastName)
        .Length(64)
        .Not.Nullable();

    HasManyToMany<Department>(x => x.DepartmentAccesses)
        .Table("EmployeeInDepartment")        
        // .ParentKeyColumn("EmployeeId")
        // .ChildKeyColumn("DepartmentId")      
        .Cascade.All();        
  }
}

Ze względu encję wtrąconą, jedna z tabel musi zostać zapisana z wartością `NULL` (identyfikator przypisywany jest w momencie fizycznego wstawienia wiersza do klucza głównego). Z założenia najpier będą tworzene departamenty.

In [13]:
using System.Data.SQLite;

var sessionFactory = CreateSessionFactory(typeof(DeskMap), typeof(DepartmentMap), typeof(EmployeeMap));

using (var session = sessionFactory.OpenSession())
{
    using (var transaction = session.BeginTransaction()) //iniialize transaction
    {
        var newton = new Department() {
            Name = "Newton",
            Address = "Kraków",
            Desks = new List<Desk>() {
                new Desk() { No = 1, Floor = 1 },
                new Desk() { No = 2, Floor = 1 },
                new Desk() { No = 3, Floor = 1 },
                new Desk() { No = 1, Floor = 2 },
                new Desk() { No = 2, Floor = 2 },
                new Desk() { No = 3, Floor = 2 },
            }
        };

        session.Save(newton);

        var pascal = new Department() {
            Name = "Pascal",
            Address = "Kraków",
            Desks = new List<Desk>() {
                new Desk() { No = 1, Floor = 4 },
                new Desk() { No = 2, Floor = 4 },                
                new Desk() { No = 1, Floor = 5 },
                new Desk() { No = 2, Floor = 5 },
                new Desk() { No = 3, Floor = 5 },
                new Desk() { No = 4, Floor = 5 },
            }
        };        

        session.Save(pascal);

        var kowalski = new Employee() 
        {
            FirstName = "Jan",
            LastName = "Kowalski",
            DepartmentAccesses = new List<Department>() { newton, pascal }
        };

        session.Save(kowalski);

        var nowak = new Employee() 
        {
            FirstName = "John",
            LastName = "Nowak",
            DepartmentAccesses = new List<Department>() { newton }
        };

        session.Save(nowak);

        transaction.Commit();
    }

    var canSit = (from c in session.Query<Employee>() //another option like linq
                 where c.LastName == "Nowak"
                 select c)
                 .FirstOrDefault();    

    foreach(var dep in canSit?.DepartmentAccesses) {
        var desks = string.Join("\n", dep.Desks.Select(d => d));
        Console.WriteLine($"John Nowak can sit in {dep.Name} on that desks:\n{desks}");
    }

    var deps = from c in session.Query<Department>()
                    select c;

    foreach(var dep in deps) {
        Console.WriteLine(dep.Employees);
        var emps = string.Join("\n", dep.Employees.Select(e => e.Id));
        Console.WriteLine($"Department {dep.Name} has employees:\n{emps}");
    }
}        


John Nowak can sit in Newton on that desks:
Desk: 1, floor: 1
Desk: 2, floor: 1
Desk: 3, floor: 1
Desk: 1, floor: 2
Desk: 2, floor: 2
Desk: 3, floor: 2



Error: System.ArgumentNullException: Value cannot be null. (Parameter 'source')
   at System.Linq.ThrowHelper.ThrowArgumentNullException(ExceptionArgument argument)
   at System.Linq.Enumerable.Select[TSource,TResult](IEnumerable`1 source, Func`2 selector)
   at Submission#13.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Query

Największą przewagą użycia *NHibernate* jest możliwość użycia *LINQ* i wszystkich jego zalet jak:
* podpowiadanie pól, 
* łatwia zmiana nazwy pola w przypadku refactoringu,
* tworzenie dynamicznych typów za pomocą `Select` i **`new`**.

Poniżej znajdują się przykładowe zapytania i sposób ich budowania.

### Klauzula **`in`**

Umożliwia operacje na kolekcjach, co umożliwia dodawanie warunków, które bazują na zbiorach danych.

In [14]:
using System.Data.SQLite;

var sessionFactory = CreateSessionFactory(typeof(DeskMap), typeof(DepartmentMap), typeof(EmployeeMap));

using (var session = sessionFactory.OpenSession())
{
    using (var transaction = session.BeginTransaction()) //iniialize transaction
    {
        var newton = new Department() {
            Name = "Newton",
            Address = "Kraków",
            Desks = new List<Desk>() {
                new Desk() { No = 1, Floor = 1 },
                new Desk() { No = 2, Floor = 1 },
                new Desk() { No = 3, Floor = 1 },
                new Desk() { No = 1, Floor = 2 },
                new Desk() { No = 2, Floor = 2 },
                new Desk() { No = 3, Floor = 2 },
            }
        };

        session.Save(newton);

        var pascal = new Department() {
            Name = "Pascal",
            Address = "Kraków",
            Desks = new List<Desk>() {
                new Desk() { No = 1, Floor = 4 },
                new Desk() { No = 2, Floor = 4 },                
                new Desk() { No = 1, Floor = 5 },
                new Desk() { No = 2, Floor = 5 },
                new Desk() { No = 3, Floor = 5 },
                new Desk() { No = 4, Floor = 5 },
            }
        };        

        session.Save(pascal);

        var kowalski = new Employee() 
        {
            FirstName = "Jan",
            LastName = "Kowalski",
            DepartmentAccesses = new List<Department>() { newton, pascal }
        };

        session.Save(kowalski);

        var nowak = new Employee() 
        {
            FirstName = "John",
            LastName = "Nowak",
            DepartmentAccesses = new List<Department>() { newton }
        };

        session.Save(nowak);

        transaction.Commit();
    }

    var names = new List<string>() { "Nowak", "Kowalski" };

    var employees = from c in session.Query<Employee>() //another option like linq
                    where names.Contains(c.LastName)
                    orderby c.LastName descending
                    select new {
                        FirstName = c.FirstName,
                        LastName = c.LastName,
                        Deps = c.DepartmentAccesses
                    };

    foreach(var emp in employees) {        
        var deps = string.Join("\n", emp.Deps.Select(x=> x.Name));
        Console.WriteLine($"Employee {emp.FirstName} {emp.LastName} has access to:\n{deps}");
    }
}

Employee John Nowak has access to:
Newton
Employee Jan Kowalski has access to:
Newton
Pascal


Dodatkowo w powyższym przykładzie użyto słowa kluczowego **`orderby`**.

### Grupowanie danych

W *Ansi SQL* oznacza użycie słowa kluczowego **`group by`**, który umożliwia tworzenie agregacji na kolekcji.

In [15]:
using System.Data.SQLite;

var sessionFactory = CreateSessionFactory(typeof(DeskMap), typeof(DepartmentMap), typeof(EmployeeMap));

using (var session = sessionFactory.OpenSession())
{
    using (var transaction = session.BeginTransaction()) //iniialize transaction
    {
        var newton = new Department() {
            Name = "Newton",
            Address = "Kraków",
            Desks = new List<Desk>() {
                new Desk() { No = 1, Floor = 1 },
                new Desk() { No = 2, Floor = 1 },
                new Desk() { No = 3, Floor = 1 },
                new Desk() { No = 1, Floor = 2 },
                new Desk() { No = 2, Floor = 2 },
                new Desk() { No = 3, Floor = 2 },
            }
        };

        session.Save(newton);

        var pascal = new Department() {
            Name = "Pascal",
            Address = "Kraków",
            Desks = new List<Desk>() {
                new Desk() { No = 1, Floor = 4 },
                new Desk() { No = 2, Floor = 4 },                
                new Desk() { No = 1, Floor = 5 },
                new Desk() { No = 2, Floor = 5 },
                new Desk() { No = 3, Floor = 5 },
                new Desk() { No = 4, Floor = 5 },
            }
        };        

        session.Save(pascal);
        
        transaction.Commit();
    }    

    var floorSits = from c in session.Query<Desk>()                        
                    group c by c.Floor into g // or simple c.Name without new
                    select new {
                        Floor = g.Key,
                        SitsCount = g.Count() // it can be used ToList() to having list of objects
                    };
                    

    foreach(var floorSit in floorSits) {                
        Console.WriteLine($"Floor {floorSit.Floor} has {floorSit.SitsCount} desks");
    }
}

Floor 1 has 3 desks
Floor 2 has 3 desks
Floor 4 has 2 desks
Floor 5 has 4 desks


### Złączenia

W czasie działania programu złączenia umożliwiają dodawanie do finalnego wyniku z bazy danych dodatkowych informacji.

## Zadania do wykonania

### Zadanie 1

Napisz program typu `CRUD` zbierający sygnały (zdarzenia) z różnych źródeł. Tabela powinna posiadać:

- dokładną datę zdarzenia,
- źródło zdarzenia,
- typ zdarzenia (informacja, ostrzeżenie, błąd, błąd krytyczny),
- dodatkowe dane,
- identyfikator (adres może być np. IP).

Na kolumnę typ zdarzenia powinien zostać założony indeks, a wartości powinny być ograniczone do tych wyliczonych w nawiasie.

### Rozwiązanie 1 zadania 1
Rozwiązanie z użyciem `NHibernate` i `SQLite`.

In [50]:
using System.Data.SQLite;
using FluentNHibernate.Cfg.Db;

public class Signal {

    public virtual int id {get; set;}
    public virtual DateTime datetime {get; set;}
    public virtual String source {get; set;}
    public virtual int type {get; set;}
    public virtual  String additionalData {get; set;}

    public Signal() {}

    public virtual String getTypeName() {
        switch(type) {
            case 0: return "information";
            case 1: return "warning";
            case 2: return "error";
            case 3: return "critical";
        }
        return null;
    }

    override
    public String ToString(){
        return String.Format(
            "{0,-24:dd-MM-yyyy HH:mm:ss.ffff} {1,-8} {2,-11} {3,-32} {4,-32}", 
            datetime, source, getTypeName(), additionalData, id);
    }

}

public class SignalMap : ClassMap<Signal> {

    public SignalMap() {
        Table("Signals");

        Id(x => x.id);
        Map(x => x.datetime);
        Map(x => x.source);
        Map(x => x.type);
        Map(x => x.additionalData);

    }

}

public class CRUDApp {

    public ISessionFactory session {get; set;}
    public FluentConfiguration config {get; set;}

    public CRUDApp(){
        connectToDatabase();
    }

    public ISessionFactory connectToDatabase(){
        if (session != null)
            return session;

        config = Fluently.Configure()
            .Database(SQLiteConfiguration.Standard
                .UsingFile("database.db")
            )
            .Mappings(m => m.FluentMappings.AddFromAssemblyOf<SignalMap>())
            .ExposeConfiguration(cfg => new SchemaUpdate(cfg).Execute(false, true));
        
        session = config.BuildSessionFactory();
        return session;
    }

    public void recreateSignalTable() {
        new SchemaExport(config.BuildConfiguration()).Execute(false, true, false);
    }

    public void createSignal(Signal signal) {
        session.OpenSession().Save(signal);
        session.Close();
    }

    public IEnumerable<Signal> readSignals() {
        var signals = session.OpenSession().Query<Signal>();
        session.Close();
        return signals;
    }

    public void updateSignal(Signal signal) {
        var session_ = session.OpenSession();
        var signal_ = session_.Get<Signal>(signal.id);
        
        signal_.datetime = signal.datetime;
        signal_.source = signal.source;
        signal_.type = signal.type;
        signal_.additionalData = signal.additionalData;
        using (ITransaction transaction = session_.BeginTransaction()){
            session_.SaveOrUpdate(signal_);
            transaction.Commit();
        }
        session_.Close();
    }

    public void deleteSignal(Signal signal) {
        var session_ = session.OpenSession();
        var signal_ = session_.Get<Signal>(signal.id);
        using (ITransaction transaction = session_.BeginTransaction()){
            session_.Delete(signal_);
            transaction.Commit();
        }

        session_.Close();
    }

}

CRUDApp program = new CRUDApp();
program.recreateSignalTable();

//Creating data
Console.WriteLine("Creating data...");

program.createSignal( new Signal() {
    datetime = DateTime.Now,
    source = "App",
    type = 1,
    additionalData = null
});

program.createSignal( new Signal() {
    datetime = DateTime.Now,
    source = "App",
    type = 1,
    additionalData = "signal to update"
});


program.createSignal( new Signal() {
    datetime = DateTime.Now,
    source = "App1",
    type = 2,
    additionalData = "signal to delete"
});


program.createSignal( new Signal() {
    datetime = DateTime.Now,
    source = "App2",
    type = 3,
    additionalData = "some other signal"
});

//Reading data
Console.WriteLine("\nReading data..");

program.readSignals().ToList().ForEach(e => Console.WriteLine(e));

//Updating data
Console.WriteLine("\nUpdating data..");
Console.WriteLine(program.readSignals().ElementAt(1));

Signal signalToUpdate = program.readSignals().ElementAt(1);
signalToUpdate.type = 0;
program.updateSignal(signalToUpdate);

Console.WriteLine(program.readSignals().ElementAt(1));

//Deleting data
Console.WriteLine("\nDeleting data..");

Signal signalToDelete = program.readSignals().ElementAt(2);
program.deleteSignal(signalToDelete);
program.readSignals().ToList().ForEach(e => Console.WriteLine(e));



Creating data...

Reading data..
07-03-2023 20:51:02.6604 App      warning                                      1                               
07-03-2023 20:51:02.6784 App      warning     signal to update                 2                               
07-03-2023 20:51:02.6932 App1     error       signal to delete                 3                               
07-03-2023 20:51:02.7072 App2     critical    some other signal                4                               

Updating data..
07-03-2023 20:51:02.6784 App      warning     signal to update                 2                               
07-03-2023 20:51:02.6784 App      information signal to update                 2                               

Deleting data..
07-03-2023 20:51:02.6604 App      warning                                      1                               
07-03-2023 20:51:02.6784 App      information signal to update                 2                               
07-03-2023 20:51:02.7072 App2     cri

### Rozwiązanie 2 zadania 1
Prymitywne podejście do korzystania z SQLa w C#

In [17]:
using System;
using System.IO;
using System.Data;
using System.Data.SqlClient;

public class EventSignal {

    public DateTime datetime {get; set;}
    public String source {get; set;}
    public int type {get; set;}
    public  String additionalData {get; set;}
    public int id {get; set;}

    public EventSignal(String source, int type, String additionalData, int id) {
        datetime = DateTime.Now;
        this.source = source;
        this.type = type;
        this.additionalData = additionalData;
        this.id = id;
    }

    public EventSignal() {}

    public String getTypeName() {
        switch(type) {
            case 0: return "information";
            case 1: return "warning";
            case 2: return "error";
            case 3: return "critical";
        }
        return null;
    }

    override
    public String ToString(){
        return String.Format(
            "{0,-24:dd-MM-yyyy HH:mm:ss.ffff} {1,-8} {2,-11} {3,-32} {4,-32}", 
            datetime, source, getTypeName(), additionalData, id);
    }

}

/*
Lokalna baza danych: SQL Server Express 2019
https://learn.microsoft.com/en-us/sql/database-engine/configure-windows/sql-server-express-localdb?view=sql-server-ver16
*/

public class CRUDApp{

    private SqlConnection connection;
    public string dir{get;}
    private SqlCommand cmd;

    public CRUDApp(){
        connection = new SqlConnection(@"Server=localhost\SQLEXPRESS01;Database=master;Trusted_Connection=True;");
        cmd = new SqlCommand();
        cmd.Connection = connection;

        dir = Directory.GetCurrentDirectory();
        cmd.CommandText = string.Format(@"
            IF NOT EXISTS
            (   SELECT * 
                FROM sys.databases 
                WHERE name = 'EventSignals' ) 
            CREATE DATABASE EventSignals ON PRIMARY
            (   NAME = EventSignals, FILENAME = '{0}\EventSignals.mdf' )
        ", dir);
        connection.Open();
        cmd.ExecuteNonQuery();
        connection.ChangeDatabase(@"EventSignals");
    }

    public void dropDatabase(){
        dropTable();
        connection.ChangeDatabase(@"master");
        cmd.CommandText = @"
            DROP DATABASE IF EXISTS EventSignals";
        cmd.ExecuteNonQuery();
        
        connection.Close();
    }

    public void disconnect() {
        connection.Close();
    }

    public void createTable() {
        try {
            cmd.CommandText = @"
                CREATE TABLE Signals (
                    datetime DATETIME,
                    source VARCHAR(8),
                    type INT,
                    additional VARCHAR(32),
                    id INT NOT NULL,
                    PRIMARY KEY (id)
                )
            ";
            cmd.ExecuteNonQuery();
        } 
        catch(SqlException) {
            Console.WriteLine("Table already exists.");
        }
    }

    public void dropTable() {
        cmd.CommandText = @"
            DROP TABLE IF EXISTS Signals";
        cmd.ExecuteNonQuery();
    }

    public void createData(EventSignal es) {
        try{
            connection.ChangeDatabase(@"EventSignals");
            cmd.CommandText = string.Format(@"
                INSERT INTO Signals (datetime, source, type, additional, id)
                VALUES ('{0}', '{1}', '{2}', '{3}', '{4}');
                ", es.datetime, es.source, es.type, es.additionalData, es.id);
            cmd.ExecuteNonQuery();
        }
        catch(SqlException e) {
            e.Display();
        }
    }

    public void createData(List<EventSignal> ess) {
        foreach (EventSignal es in ess)
            createData(es);
    }

    public EventSignal readData(int id) {
        var datatable = new DataTable("Signals");

        try {
            cmd.CommandText = string.Format(@"
                SELECT * FROM Signals WHERE id={0}
            ", id);
            new SqlDataAdapter(cmd).Fill(datatable);
            var row = datatable.Rows[0];
            var es = new EventSignal() {
                datetime = DateTime.Parse(row.ItemArray[0].ToString()),
                source = row.ItemArray[1].ToString(),
                type = int.Parse(row.ItemArray[2].ToString()),
                additionalData = row.ItemArray[3].ToString(),
                id = int.Parse(row.ItemArray[4].ToString())
            };
            return es;
        }
        catch(SqlException e) {
            e.Display();
        }

        return null;
    }

    public List<EventSignal> readData() {
        var datatable = new DataTable("Signals");
        var ess = new List<EventSignal>();

        try {
            cmd.CommandText = @"
                SELECT * FROM Signals
            ";
            new SqlDataAdapter(cmd).Fill(datatable);
            foreach(DataRow row in datatable.Rows) {
                ess.Add(
                    new EventSignal() {
                        datetime = DateTime.Parse(row.ItemArray[0].ToString()),
                        source = row.ItemArray[1].ToString(),
                        type = int.Parse(row.ItemArray[2].ToString()),
                        additionalData = row.ItemArray[3].ToString(),
                        id = int.Parse(row.ItemArray[4].ToString())
                    }
                );
            }
            return ess;
        }
        catch(SqlException e) {
            e.Display();
        }
        return new List<EventSignal>();
    }

    public void updateData(EventSignal es) {
        try {
            cmd.CommandText = string.Format(@"
                UPDATE Signals 
                SET 
                    datetime = '{1}',
                    source = '{2}',
                    type = '{3}',
                    additional = '{4}'
                WHERE id={0}
            ", es.id, es.datetime, es.source, es.type, es.additionalData);
            cmd.ExecuteNonQuery();
        }
        catch(SqlException e) {
            e.Display();
        }
    }

    public void deleteData(int id) {
        try {
            cmd.CommandText = string.Format(@"
                DELETE FROM Signals
                WHERE id={0}
            ", id);
            cmd.ExecuteNonQuery();
        }
        catch(SqlException e) {
            e.Display();
        }
    }

}

/*  
    Example
    W tym przykładzie zsotanie utworzona baza danych,
    do której zostanie zapisana lista sygnałów.
    Bezpośrednio na bazie lokalnej zostanie wykonana 
    operacja uaktualniania oraz usuwania danych.
    Po każdej z tych operacji zmiany będą widoczne w konsoli.

    Dla celów demonstracyjnych, każdorazowe uruchomienie
    skryptu powoduje utworzenie bazy danych, tabeli,
    wykonanie celów zadania oraz usunięcie bazy danych i
    zamknięcie połączenia.

    Jako serwer lokalny bazy danych wykorzystaliśmy SQLEXPRESS.
*/

var db = new CRUDApp();
db.createTable();

//Creating data
Console.WriteLine("Creating data...");
var evsToUpload = new List<EventSignal> {
    new EventSignal("App", 1, null, 0),
    new EventSignal("App", 1, "another one", 1),
    new EventSignal("App1", 2, "signal to update", 2),
    new EventSignal("App2", 3, "do not delete", 3),
};
db.createData(evsToUpload);

//Reading data
Console.WriteLine("\nReading data...");
foreach (EventSignal es in db.readData()){
    Console.WriteLine(es.ToString());
}

//Updating data
Console.WriteLine("\nUpdating data...");
var evToUpdate = db.readData(2);
evToUpdate.type = 0;
db.updateData(evToUpdate);

Console.WriteLine( db.readData(2).ToString() );

//Deleting data
Console.WriteLine("\nDeleting data...");
db.deleteData(0);
db.deleteData(1);

foreach (EventSignal es in db.readData()){
    Console.WriteLine(es.ToString());
}

db.dropDatabase();
db.disconnect();

Creating data...

Reading data...
03-07-2023 18:57:24.0000 App      warning                                      0                               
03-07-2023 18:57:24.0000 App      warning     another one                      1                               
03-07-2023 18:57:24.0000 App1     error       signal to update                 2                               
03-07-2023 18:57:24.0000 App2     critical    do not delete                    3                               

Updating data...
07-03-2023 18:57:24.0000 App1     information signal to update                 2                               

Deleting data...
07-03-2023 18:57:24.0000 App1     information signal to update                 2                               
03-07-2023 18:57:24.0000 App2     critical    do not delete                    3                               


### Zadanie 2

Do zadania 2 z laboratorium 1 dodaj obsługę bazy danych.

In [92]:
using System.Collections.Generic;
using System.IO;
using System.Data.SQLite;
using FluentNHibernate.Cfg.Db;
using FluentNHibernate.Mapping;


public class Pracownik {

    public virtual String imie {get; set;}
    public virtual String nazwisko {get; set;}
    public virtual int wiek {get; set;}
    public virtual String stanowisko {get; set;}
    public virtual String plec {get; set;}

    // walidacja czy wszystkie dane są wprowadzone
    public virtual bool Validate(){
        if (string.IsNullOrEmpty(imie) || string.IsNullOrEmpty(nazwisko) || wiek == 0 || string.IsNullOrEmpty(stanowisko)) {
            Console.WriteLine("Pracownik zawiera puste atrybuty");
            Console.WriteLine(Show());
            return false;
        }
        return true;
    }

    // metoda show wyświetlająca pracownika
    public virtual string Show() {
        return string.Format(
            "| {0,-15}| {1,-15}| {2,-4}| {3,-15}| {4,-2}|",
            imie, nazwisko, wiek, stanowisko, plec);
    }

    // sprawdzenie czy istnieje juz pracownik
    public virtual bool IsMatch(Pracownik pracownik){
        if( imie == pracownik.imie && 
            nazwisko == pracownik.nazwisko &&
            wiek == pracownik.wiek &&
            stanowisko == pracownik.stanowisko &&
            plec == pracownik.plec)
                return true;
            else return false;
    }

}

//Klasa generyczna kartoteka, pracownicy składowani w liście
public class Kartoteka<T> where T : Pracownik{

    public List<T> baza {get; set;} = new List<T>();

    public void Dodaj(T pracownik) {
        if(pracownik.Validate() == true && IsExist(pracownik)) baza.Add(pracownik);
    }

    public void Usun(T pracownik) {
        baza.Remove(pracownik);
    }

    // czy juz jest obiekt o takich samych atrybutach
    public bool IsExist(T pracownik) {
        if (baza.Any(x => x.IsMatch(pracownik) == true)) {
            Console.WriteLine("\n Istnieje juz taki pracownik");
            Console.WriteLine(pracownik.Show());
            return false;
        }
        else return true;
    }

    // do wyswietlania
    public void Wyswietl() {
        Console.WriteLine(
            string.Format(
                " {0,-15}  {1,-15}  {2,-4}  {3,-15}  {4,-2}", 
                "imie", "nazwisko", "wiek", "stanowisko", "plec"));

        foreach(T a in baza) {
            Console.WriteLine(a.Show());
        }
    }

    public void Szukaj(String name) {
        Console.WriteLine("\nPracownik o imieniu:  \"{0}\":", name.ToLower());
        foreach(T a in baza) {
            if (a.Show().ToLower().Contains(name.ToLower()))
            {
                Console.WriteLine(a.Show());
            }
        }
    }    

    public bool Save(IBuilder builder) {
        return builder.Save(this);
    }

    public bool Read(IBuilder builder) {
        var loaded = builder.Read("Kartoteka.json");
        return true;
    }

    public interface IBuilder {
        bool Save(Kartoteka<T> baza);
        Kartoteka<T> Read(String fileName);
    }
    
}

public class PracownikMap : ClassMap<Pracownik> {
    public PracownikMap() {
        Table("Employees");

        Map(x => x.imie);
        Id(x => x.nazwisko);
        Map(x => x.wiek);
        Map(x => x.stanowisko);
        Map(x => x.plec);
    }

}

public class SQLBuilder : Kartoteka<Pracownik>.IBuilder {

    public ISessionFactory session {get; set;}
    public FluentConfiguration config {get; set;}

    public SQLBuilder() {
        connectToDatabase();
    }

    public ISessionFactory connectToDatabase(){
        if (session != null)
            return session;

        config = Fluently.Configure()
            .Database(SQLiteConfiguration.Standard
                .UsingFile("database2.db")
            )
            .Mappings(m => m.FluentMappings.AddFromAssemblyOf<PracownikMap>())
            .ExposeConfiguration(cfg => new SchemaUpdate(cfg).Execute(false, true));
        
        session = config.BuildSessionFactory();
        return session;
    }

        public void recreateSignalTable() {
        new SchemaExport(config.BuildConfiguration()).Execute(false, true, false);
    }
    
    public bool Save(Kartoteka<Pracownik> baza) {
        if (typeof(Kartoteka<Pracownik>).Equals(baza.GetType())) {
            var session_ = session.OpenSession();
            var transaction = session_.BeginTransaction();
            foreach(Pracownik emp in baza.baza.ToList()){
                session_.Save(emp);
            }
            transaction.Commit();
            session_.Close();
            return true;
        }
        return false;
    }

    public Kartoteka<Pracownik> Read() {
        var emps = session.OpenSession().Query<Pracownik>();
        session.Close();
        return new Kartoteka<Pracownik>() { baza = emps.ToList() };
    }

    public Kartoteka<Pracownik> Read(String name) {
        return Read();
    }

}

var kartoteka = new Kartoteka<Pracownik>();
kartoteka.Dodaj(new Pracownik(){imie = "Szymon", nazwisko = "Zielinski", wiek = 21, stanowisko = "Employee", plec = "m"});
kartoteka.Dodaj(new Pracownik(){imie = "Patryk", nazwisko = "Pirog", wiek = 23, stanowisko = "Manager", plec = "m"});
kartoteka.Dodaj(new Pracownik(){imie = "Jan", nazwisko = "Kowalski", wiek = 28, stanowisko = "Employee", plec = "m"});

Console.WriteLine("\n KARTOTEKA:");
kartoteka.Wyswietl();

var sqlbuilder = new SQLBuilder();
sqlbuilder.recreateSignalTable();

Console.WriteLine();
Console.WriteLine(@"Zapisano do bazy danych: {0}", kartoteka.Save(sqlbuilder));

var drugaKartoteka = sqlbuilder.Read();
drugaKartoteka.Wyswietl();


 KARTOTEKA:
 imie             nazwisko         wiek  stanowisko       plec
| Szymon         | Zielinski      | 21  | Employee       | m |
| Patryk         | Pirog          | 23  | Manager        | m |
| Jan            | Kowalski       | 28  | Employee       | m |

Zapisano do bazy danych: True
 imie             nazwisko         wiek  stanowisko       plec
| Szymon         | Zielinski      | 21  | Employee       | m |
| Patryk         | Pirog          | 23  | Manager        | m |
| Jan            | Kowalski       | 28  | Employee       | m |


### Zadanie 3

Napisz klasę do logowania błędów. Zdarzenia powinny być logowane do bazy danych jako *bulk load*. Klasa powinna obsługiwać dwie bazy danych - główną (dowolna relacyjna baza danych) i lokalna (na dane tymczasowe). Schemat logowania jest następujący:
- jeśli baza danych (operacyjna) jest niedostępna logowanie powinno nastąpić do lokalnej bazy danych,
- jeśli połączenie do głównej bazy głównej zostanie przywrócone, klasa powinna przenieść wszystkie dane z lokalnej bazy danych do głównej.

Należy pamiętać o unikalności klucza głównego.

In [19]:
using System;
using System.Data.SqlClient;
using System.Data;

class ErrorLogger {
    private SqlConnection mainDbConnection;
    private SqlConnection localDbConnection;
    private DataTable localErrorLog;

    public ErrorLogger(string mainDbConnectionString, string localDbConnectionString) {
        mainDbConnection = new SqlConnection(mainDbConnectionString);
        localDbConnection = new SqlConnection(localDbConnectionString);
        localErrorLog = new DataTable();
        localErrorLog.Columns.Add("Date", typeof(DateTime));
        localErrorLog.Columns.Add("Source", typeof(string));
        localErrorLog.Columns.Add("Type", typeof(string));
        localErrorLog.Columns.Add("AdditionalData", typeof(string));
        localErrorLog.Columns.Add("ID", typeof(string));
        localErrorLog.PrimaryKey = new DataColumn[] { localErrorLog.Columns["Identifier"] };
    }

    public void LogError(DateTime errorDate, string errorSource, string errorType, string additionalData, string identifier) {
        try {
            using (SqlCommand command = new SqlCommand("INSERT INTO ErrorLog (Date, Source, Type, AdditionalData, ID) VALUES (@Date, @Source, @Type, @AdditionalData, @ID)", mainDbConnection)) {
                command.Parameters.AddWithValue("@Date", errorDate);
                command.Parameters.AddWithValue("@Source", errorSource);
                command.Parameters.AddWithValue("@Type", errorType);
                command.Parameters.AddWithValue("@AdditionalData", additionalData);
                command.Parameters.AddWithValue("@ID", identifier);
                command.ExecuteNonQuery();
            }
        }
        catch (SqlException) {
            localErrorLog.Rows.Add(errorDate, errorSource, errorType, additionalData, identifier);
        }
    }

    public void CheckMainDbConnection() {
        if (mainDbConnection.State == ConnectionState.Closed) {
            mainDbConnection.Open();
        }

        if (mainDbConnection.State == ConnectionState.Open) {
            using (SqlBulkCopy bulkCopy = new SqlBulkCopy(mainDbConnection)) {
                bulkCopy.DestinationTableName = "ErrorLog";
                bulkCopy.WriteToServer(localErrorLog);
            }
            localErrorLog.Clear();
        }
    }
}